In [ ]:
#USING EXCEL

import pandas as pd
from transformers import pipeline

# Load the Excel file (update the file path as needed)
file_path = '/content/allottedlist.xlsx'
data = pd.read_excel(file_path)

# Load model and tokenizer
model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

# Function to create context from data
def create_context(data):
    context = ""
    for index, row in data.iterrows():
        context += f"Rank: {row['Rank']}, Quota: {row['Allotted Quota']}, Institute: {row['Allotted Institute']}, Category: {row['Allotted Category']}. "
    return context

# Define the function to get the prediction
def get_prediction(question, context):
    QA_input = {
        'question': question,
        'context': context
    }
    res = nlp(QA_input)
    return res['answer']

# Function to predict the allotted institute based on rank, quota, and category
def predict_allotted_institute(rank, quota, category):
    filtered_data = data[(data['Rank'] == int(rank)) & (data['Allotted Quota'] == quota) & (data['Allotted Category'] == category)]
    if not filtered_data.empty:
        institute = filtered_data['Allotted Institute'].values[0]
        return institute
    else:
        return "No matching institute found"

# Function to predict the rank required for a specific institute, quota, and category
def predict_rank_for_institute(institute, quota, category):
    filtered_data = data[(data['Allotted Institute'] == institute) & (data['Allotted Quota'] == quota) & (data['Allotted Category'] == category)]
    if not filtered_data.empty:
        return filtered_data['Rank'].min()  # Assuming you want the best rank for the institute
    else:
        return "No matching rank found"

# Function to find all possible colleges for a given rank, quota, and category
def find_possible_colleges(rank, quota, category):
    filtered_data = data[(data['Rank'] <= int(rank)) & (data['Allotted Quota'] == quota) & (data['Allotted Category'] == category)]
    if not filtered_data.empty:
        unique_colleges = filtered_data.groupby('Allotted Institute')['Rank'].min().reset_index()
        sorted_colleges = unique_colleges.sort_values(by='Rank')
        return sorted_colleges
    else:
        return None

# List of all quota options
quota_options = [
    'Open Seat Quota',
    'All India',
    'Delhi University Quota',
    'IP University Quota',
    'Deemed/Paid Seats Quota',
    'Internal - Puducherry UT Domicile',
    'Aligarh Muslim University (AMU) Quota',
    'Delhi NCR Children/Widows of Personnel of the Armed Forces (CW) Quota',
    'Employees State Insurance Scheme (ESI)',
    'Foreign Country Quota',
    'Non-Resident Indian',
    'Muslim Minority Quota',
    'Jain Minority Quota',
    'Non-Resident Indian(AMU)Quota',
    'Employees State Insurance Scheme(ESI)'
]

# List of all category options
category_options = [
    'Open',
'OBC',
'SC',
'EWS',
'ST',
'Open PwD',
'OBC PWD',
'EWS PWD',
'SC PwD',
'ST PwD'
]

# List of all institute options
institute_options = [
    'JIPMER Puducherry',
'AIIMS, New Delhi',
'Institute of PG Medical Educational and Research, Kolkata',
'Madras Medical College, Chennai',
'AIIMS, Bhubaneswar',
'Atal Bihari Vajpayee Institute of Medical Sciences and Dr RML Hospital',
'Seth G.S. Medical College, Mumbai',
'Maulana Azad Medical College',
'Vardhman Mahavir Medical College and Safdarjung Hospital',
'AIIMS, Jodhpur',
'M.P. Shah Medical College, Jamnagar',
'University College of Medical Sciences',
'S.M.S. Medical College, Jaipur',
'Government Medical College And Hospital, Chandigarh',
'AIIMS, Bhopal',
'AIIMS, Rishikesh',
'Jawaharlal Nehru Medical College, AMU, Aligarh',
'Government Medical College Baroda, Anandpura',
'Institute of Medical Sciences, Varanasi',
'Lady Hardinge Medical College (Female Seat only)',
'AIIMS, Nagpur',
'Government Medical College, Thiruvananthapuram',
'Dr. B.S.A. Medical College',
'Government Medical College, Surat',
'B.J. Medical College, Ahmedabad',
'AIIMS Mangalagiri',
'Kasturba Medical College, Manipal',
'Government Medical College, Kottayam',
'Bangalore Medical College and Research Institute',
'Gandhi Medical College, Secunderabad',
'Karnatak Institute Of Medical Science, Hubli',
'B.J. Government Medical College, Pune',
'AIIMS, Patna',
'KGMC, Lucknow',
'AIIMS, Kalyani',
'Medical College, Kolkata',
'Indira Gandhi Medical College',
'AIIMS, Raipur',
'Government Medical College, Srinagar',
'AIIMS, Bibi Nagar',
'Government Medical College, Kozhikode',
'Coimbatore Medical College, Peelamedu',
'AIIMS Rajkot',
'Lokmanya Tilak Municipal Medical College, Mumbai',
'Madurai Medical College, Madurai',
'NDMC Medical College',
'AIIMS Bathinda',
'Stanley Medical College, Chennai',
'Grant Medical College and Sir J.J.Hospital, Mumbai',
'AIIMS Guwahati',
'JIPMER Karaikal',
'Belgaum Institute Of Medical Science, Belagavi',
'AIIMS Bilaspur Changar Palasiyan',
'Sardar Patel Medical College, Bikaner',
'S.C.B. Medical College, Cuttack',
'Indira Gandhi Institute of Medical Sciences, Patna',
'AIIMS, Gorakhpur',
'R.N.T. Medical College, Udaipur',
'Dr. Ram Manohar Lohia Institute of Medical Science, Lucknow',
'L.L.R.M. Medical College, Meerut',
'M.G.M. Medical College, Indore',
'AIIMS, Rai Bareli',
'Government Medical College, Thrissur',
'DR.S.N. Medical College, Jodhpur',
'Pt. D.D.U Medical College, Rajkot',
'T.D. Medical College, Allappuzha',
'Mysore Medical and Research Institute, Mysore',
'Pt. B.D. Sharma PGIMS',
'Government Medical College, Patiala',
'Dr. B.R. Ambedkar State Institute of Medical Sciences',
'AIIMS Jammu',
'Government Medical College, Manjeri',
'Netaji Subhash Chandra Bose Mc, Jabalpur',
'Guntur Medical College, Guntur',
'Topiwala National Medical College, Mumbai',
'Patna Medical College',
'Kurnool Medical College, Kurnool',
'Osmania Medical College, Koti, Hyderabad',
'Jawaharlal Nehru Medical, Ajmer',
'Government Medical College, Kota',
'Atal Bihari Vajpayee Government Medical College, Vidisha',
'Shri Atal Bihari Vajpayee Medical College and Research Institute, Bengaluru',
'Nilratan Sirkar Medical College, Kolkata',
'Government Kilpauk Medical College, Chennai',
'RUHS College of Medical Sciences, Jaipur',
'Government Medical College, Ernakulam',
'AIIMS, Deoghar',
'Hinduhridayasamrat Balasaheb Thackeray Medical College and Dr. R. N. Cooper Municipal General Hospital, Juhu',
'Gajra Raja Medical College, Gwalior',
'Siddartha Medical College, Vijayawada',
'Calcutta National Medical College, Kolkata',
'ESIC Medical College, Faridabad',
'Goa Medical College, Panaji',
'ESI-MC and PGIMS and R, Bangalore',
'Government Medical College, Aurangabad',
'G.S.V.M. Medical College, Kanpur',
'Maharaja K.C. Gajapati Medical College, Brahmapur',
'Indira Gandhi Medical College and RI, Puducherry',
'ESIC Medical College, Hyderabad',
'Government Vellore Medical College, Vellore',
'Government Medical College, Nagpur',
'Rajendra Institute of Medical Sciences, Ranchi',
'Pt. J N M Medical College, Raipur',
'AIIMS, Madurai',
'Andhra Medical College, Visakhapatnam',
'Indira Gandhi Government Medical College, Nagpur',
'S.N. Medical College, Agra',
'Maharani Laxmi Bai Medical College, Jhansi',
'Government Medical College, Omandurar',
'M.G.M. Medical College, Jamshedpur',
'DR.Rajendra Prasad Medical College, Tanda',
'R G Kar Medical College',
'Dr.S.C. Government Medical College, Nanded',
'Government Institute of Medical Sciences, Greater Noida',
'Government Medical College, ESIC, Kollam',
'Government Medical College and ESIC Hospital, Coimbatore',
'Moti Lal Nehru Medical College, Allahabad',
'Government medical college, Amritsar',
'Gandhi Medical College, Bhopal',
'MG Institute of Medical Sciences, Sevagram Wardha',
'Government Medical College, Jammu',
'Rangaraya Medical College, Kakinada',
'Guwahati Medical College, Guwahati',
'K.A.P. Viswanatham Government Medical College, Tiruchirapalli',
'Rajiv Gandhi Medical College, Thane',
'Gulbarga Institute of Medical Sciences',
'Government Doon Medical College, Dehradun',
'Government Medical College, Tirunelveli',
'V.S.S. Medical College, Burla',
'Government Medical College, Kanker',
'Guru Govind Singh Medical College',
'B.R.D. Medical College, Gorakhpur',
'Government Medical College, Palakkad',
'Uttar Pradesh University of Medical Sciences, Saifai',
'Kalpana Chawla Government Medical College',
'Government Medical College, Miraj',
'Nalanda Medical College, Patna',
'Rajarshee Chhatrapati Shahu Maharaj Government Medical College Kolhapur',
'Vijayanagar Institute of Medical Science, Bellary',
'SLBS Government Medical College, Mandi',
'Uttaranchal F Hospital Trust Medical College, Haldwani',
'Andaman and Nicobar Islands Institute of Medical Science',
'NEIGRIHMS, Shillong',
'Mandya Institute Of Medical Science, Mandya',
'Medical College, Bhavnagar',
'Sagar Dutta Medical College and Hospital',
'Regional Inst Of Medical Sci, Imphal',
'Government Medical College, Kannur',
'Dr. YS Parmar Government Medical College, Sirmaur',
'Jhalawar Medical College, Jhalawar',
'Barasat Government Medical College and Hospital',
'Government Medical College, Idukki',
'Jawaharlal Nehru Medical College, Bhagalpur',
'Vardhman Institute Of Medical Sciences, Nalanda',
'ACSR Government Medical College, Nellore',
'North Bengal Medical College, Darjeeling',
'Employees State Insurance Corporation Medical College, Alwar',
'Karwar Institute of Medical Sciences, Karwar',
'Darbhanga Medical College, Laheriasarai',
'ESIC Medical College and PGIMSR, Chennai',
'Government Medical College, Pali',
'Government Medical College Baramati',
'Shaheed Nirmal Mahto Medical College and Hospital, Dhanbad',
'Shri Kalyan Government Medical College, Bajaj Gram',
'Sri Venkateswara Medical College, Tirupati',
'Government Medical College, Bhilwara',
'Burdwan Medical College, Burdwan',
'Government Medical College, Konni',
'ESIC PGIMSR, Joka',
'SJP Medical College, Bharatpur',
'Government Medical College and Super Facility Hospital, Azamgarh',
'Government Medical College, Churachandpur',
'Chengalpattu Medical College, Kancheepuram',
'Bankura Sammilani Medical College, Lokepur',
'College of Medicine and JNM Hospital, Kalyani',
'Dr. Radhakrishnan Government Medical College, Hamirpur',
'Government medical college, Dausa Rajasthan',
'BPS Government Medical College, Sonepat (Female Seat only)',
'Government Medical College, Dungarpur',
'Government Medical college, Sriganganagar',
'Hassan Institute Medical Sciences, Hassan',
'Kakatiya Medical College, Warangal',
'Silchar Medical College, Silcher',
'Shri Atal Bihari Vajpayee Government Medical College',
'Chhattisgarh Institute of Medical Sciences, Bilasp',
'Bundelkhand Medical College, Sagar',
'Mursidabad Medical College and Hospital, Mursidabad',
'Rajkiya Medical College, Jalaun',
'Autonomous State Medical College, Siddharthnagar',
'Thanjavur Medical College,Thanjavur',
'Dr. Vaishampayam Memorial Medical College, Sholapur',
'SUH Maulana Mahmood Hasan Medical College, Saharanpur',
'Pt. Jawahar Lal Nehru Government Medical College, Chamba',
'Assam Medical College, Dibrugarh',
'Government Medical College Chittorgarh',
'Sri Krishna Medical College, Muzaffarpur',
'S.S. Medical College, Rewa',
'Government Medical College Firozabad,Daulattapur',
'Veer Chandra Singh Garhwali Government Institute of Medical Science and Research,Badrinath Marg',
'SKIMS Medical College, Bemina',
'Kodagu Institute of Medical Sciences, Kodagu',
'Government Medical College Faizabad',
'Shimoga Institute Of Medical Science, Shimoga',
'Jorhat Medical College and Hospital',
'College Government Medical College Alwar',
'Midnapore Medical College, Midnapur',
'Government Medical College And General Hospital, Satara',
'Rajiv Gandhi Institute Of Medical Sciences, Kadapa',
'Autonomous State Medical College Society , Mirzapur',
'Government Medical College Mahasamund',
'Anugrah Narayan Magadh Medical College, Gaya',
'ESIC Medical College, Gulbarga',
'ESIC Medical College and Hospital, Bihta',
'MRA Medical College , Ambedkar nagar',
'Soban Singh Jeena Government Institute of Medical Science and Research, Almora',
'Kanyakumari Government Medical College,Asaripallam',
'Government Medical College, Akola',
'Government Medical College Barmer Rajasthan',
'Government Medical College, Churu',
'Government Mohan Kumaramangalam Medical College, Salem',
'Government Medical College, Kannauj',
'Goverment Medical College, Datia',
'Government Medical College Basti,Rampur',
'Theni Government Medical College, Theni',
'Government Medical College Ratlam,Gram Banjali',
'Raichur Institute Of Medical Science, Raichur',
'Government Medical College Karimnagar',
'Shaheed Hasan Khan, Mewati Government Medical College',
'Government Erode Medical College Perundurai',
'Government Medical College Badaun',
'Autonomous State Medical College, Pratapgarh',
'UNS Autonomous State Medical Colleges, Jaunpur',
'Government Medical College Nalgonda',
'Government Medical College Satna',
'Government Medical College, Thiruvallur',
'Sh Vasant Rao Naik Government Medical College ,Yavatmal',
'Government medical college Karauli',
'Sri Jagannath Medical College and Hospital, Puri',
'Gvmc, Villupuram',
'Government Medical College, Latur',
'Lt. B R K Government Medical College, Jagdalpur',
'Diamond Harbour Government Medical College,New Town',
'Government Medical College, Nilgiris',
'Saheed Laxman Nayak Medical College and Hospital, Koraput',
'Bidar Institute of Medical Science',
'Government Medical College and Hospital, Chandrapur',
'Government Medical College, Dholpur',
'Government Medical college Shivpuri',
'Chhindwara Institute of Medical Sciences, Chhindwara',
'Government Pudukkottai Medical College Hospital, Pudukkottai',
'Rajah Muthiah Medical College, Annamalai',
'Government Medical College, Dindigul',
'Government Medical College, Sirohi',
'Thoothukudi Medical College,Thoothukudi',
'Autonomous State Medical College ,Etah',
'Government medical college, Shahjhanpur',
'Fakhruddin Ali Ahmed Medical College, Barpeta',
'Government Medical College, Khandwa',
'Rani Durgavati Medical College, Banda',
'Goverment Medical College, Bettiah',
'Government Dharamapuri Med Coll,Dharmapuri',
'Shri Bhausaheb Hire Government Medical College, Dhule',
'Hamdard Inst of Med Sciences and Research',
'Goverment Medical College And Hospital Jalgaon',
'Maharshi Devraha Baba Autonomous State. Medical College, Deoria',
'Goverment Medical College Nandurbar',
'Sheikh Bhikhari Medical College and Hospital, Hazaribag ( Formerly Called As- Hazaribagh Medical College, Hazaribag)',
'Government Medical College, Ananthapuram',
'Government Medical College, Tiruppur',
'Government Medical College, Krishnagiri',
'Phulo Jhano Medical College , Dumka',
'Malda Medical College',
'Lt. L A M Government Medical College, Raigarh',
'Government Medical College, Gondia',
'SVIMS - Sri Padmavathi Medical College for Women, Tirupati (Female Seat only)',
'Bhima Bhoi Medical College and Hospital , Balangir',
'Chandulal Chandrakar Memorial Government Medical College, Durg',
'Government Medical College and District General Hospital,Ratnagiri',
'Government medical College, Namakkal',
'Autonomous State Medical college Society Hardoi,Gaura Danda',
'Agartala Government Medical College,Agartala',
'Government Medical College and Hospital, Alibag-Raigad',
'Medinirai Medical College (previously Known as Palamu Medical College)',
'Deben Mahata Government Medical College and Hospital,Vill Hatuara',
'Autonomous State Medical College , Fatehpur',
'Government Medical College Karur',
'Rajiv Gandhi Institute Medical Science of Adilabad',
'Swami Ramanand Tirth Rural Medical College,Ambajogai',
'Government Medical College and Hospital, Jalpaiguri',
'Rampurhat Government Medical College Rampurhat',
'GMERS Medical College, Porbandar',
'GMERS Medical College, Navsari',
'Government Thiruvarur medical college, Thiruvarur',
'Jannayak Karpoori Thakur Medical college and Hospital, Madhepura',
'Gadag Institute of Medical Sciences',
'Maharaja Jitendra Narayan Medical College and Hospital Coochbehar',
'Government of Medical College and Hospital, Balasore',
'Government Medical College, Kathua',
'Chikkaballapura Institute of Medical Sciences',
'Raiganj Government Medical College',
'Lakhimpur Medical College',
'Government Medical College, Korba',
'Sarat Chandra Chattopadhyay Government Medical College and Hospital, Uluberia',
'JLN IMS, IMPHAL',
'Government Medical College Shahdol',
'Autonomous State Medical College, Ghazipur',
'Government Medical College, Rajnandgaon',
'Saheed Rendo Majhi Medical College and Hospital, Bhawanipatna',
'Tezpur Medical College, Tezpur',
'Government Sivgangai Medical College Sivagangai',
'Government Medical College Purnea',
'Pt. Raghunath Murmu Medical College, Baripada',
'GMERS Medical College, Morbi',
'Government Medical College Jayashankar Bhupalpally',
'GMERS Medical College, Panchmahal Godhra',
'Thiruvannamalai Medical College, Thiruvannamalai',
'Government Medical College, Osmanabad',
'Dhubri Medical College',
'Government Medical College, Mahabubangar',
'Government Medical College, Virudhunagar',
'Government Medical College, Doda',
'Government medical college Jangaon',
'Government Medical College Rajanna Sircilla',
'Rajmata shrimati devendra kumari singhdeo government medical college, ambikapur',
'Government Medical College Siddipet',
'Government Medical College, Ramanathapuram',
'Haveri Institute of Medical Sciences, Karnataka',
'Prafulla Chandra Sen Government Medical College and Hospital, Arambagh',
'RIMS Srikakulam',
'Government Medical College, Parbhani',
'Government Medical College, Wanaparthy',
'Government Medical College, Baramulla',
'Government Medical College Khaleelwadi',
'Government Medical College, Ariyalur',
'Government Medical College and Hospital, Keonjhar',
'Government Medical College, Kheora',
'Government Medical College, Nagarkurnool',
'Government Medical College, Kallakurichi',
'C. Institute of Medical Sciences, Chamarajanagar',
'Nalbari Medical College and Hospital Dakhingaon',
'Government Medical College, Handwara',
'RIMS, Ongole',
'Koppal Institute of Medical Sciences',
'GMERS Medical College, Rajpipla',
'Government Medical College, Sindhudurg',
'Government Medical College, Mancherial',
'Tamralipto Government Medical College and Hospital',
'Jhargram Government Medical College and Hospital',
'Government Medical College Machilipatnam',
'Nagaland Institute Of Medical Science And Research Phirebagie',
'Government Medical College, Khammam',
'Chikkamagaluru Institute of Medical Sciences',
'Government Medical College, Eluru',
'Zoram Medical College',
'Government Medical College, Mahabubabad',
'Government Medical College Suryapet',
'Yadgiri Institute Of Medical Sciences, Yadgiri',
'Tomo Riba Institute Health and Medical Sciences, Naharlagun',
'Diphu Medical College and Hospital',
'Government Medical College, Sundargarh',
'Government Medical College, Sangareddy',
'Government Medical College, Nagapattinam',
'Government medical college Kamareddy',
'Nagaon Medical college,Dipholu',
'Government Medical College Nandyal',
'Government Medical College Vikarabad',
'Kokrajhar Medical College and Hospital Rangalikhata',
'Government medical college, Rajamahendravaram',
'Government Medical College, Jagtial',
'Government Medical College, Ramagundam',
'Government Medical College, Bhadradri',
'Government Medical College Vizianagaram',
'Kasturba Medical College, Mangalore',
'Government Medical College Nirmal',
'Sri Ramachandra Medical College and Research Institute, Chennai',
'Government Medical College, Kumuram Bheem Asifabad',
'Amrita School of Medicine Faridabad',
'Symbiosis Medical College for Women, Pune (Female Seat only)',
'JLN Medical College, Datta Meghe',
'MGM Medical College, Navi Mumbai',
'Yenepoya Medical College, Mangalore',
'Manipal Tata Medical College, Jamshedpur',
'Amrita Institute of Medical Science, Kochi',
'JSS Medical College, Mysuru',
'SBKS Medical Institute and Research Centre, Sumandeep Vidyapeeth',
'Jawahar Lal Nehru Medical College, Belagavi',
'Kalinga Institute of Medical Sciences, Bhubaneswar',
'M.M Institute of medical and Research',
'K.S Hegde Medical Academy, Mangaluru',
'MGM Medical College, Aurangabad',
'Santosh Medical College and Hospital',
'Bharati Vidyapeeth DU Medical College,Dhankawadi',
'Rural Medical College and PIMS, Loni',
'Dr. DY Patil Medical College and Hospital, Pune',
'Raja Rajeswari Medical College Bengaluru',
'Dr. DY Patil Medical College, Navi mumbai',
'Institute of Medical Sciences and SUM Hospital, Bhubaneswar',
'Aarupadai Veedu Medical College and Hospital',
'Sree Balaji Medical College and Hospital, Chennai',
'Jagadguru Gangadhar Mahaswamigalu Moorusavirmath Medical College, Hubballi',
'Datta Meghe Medical College, Wanadongri',
'Krishna Institute of Medical Science, Karad',
'Mahatma Gandi Mission Medical College, vashi',
'Dr. DYP Education Society Deemed University, Kolhapur',
'SDU Medical College, Kolar',
'B.L.D.E University, Bijapur',
'Bhaarath Medical College And Hospital, Chennai',
'Saveetha Medical College, Chennai',
'Sri Lakshmi Narayana Institute of Medical Science',
'Sri Siddhartha Medical College DU, Tumkur',
'SRM Medical College and Hospital, Chennai',
'VMKV Medical College and Hospital, Salem',
'Sri Lalithambigai Medical College and Hospital',
'Chettinad Hospital and Research Institute, Kancheepuram',
'BV Deemed University Medical College and Hospital, Sangli',
'GITAM Institue of Medical Science and Research, Visakhapatnam',
'Mahatma Gandhi Medical College, Pondicherry',
'Vinayaka Missions Medical College and Hospital, Karaikal',
'VELS Medical College and Hospital, Tiruvallur',
'ACS Medical College and Hospital, Chennai',
'Sri Siddhartha institute of Medical Science and Research Centre, T Begur',
'Shri Sathya Sai Medical College and RI, Chennai',
'NAMO Medical Education and Research Institute, Govt Medical College Silvassa',
'Government Medical College, Hanumangarh',
'Government medical college, Bahraich',
'Government Medical College, Bundi',
'Government Medical College, Udhampur',
'Government Medical College, Anantnag',
'Meenakshi Medical College Hospital and Research Institute, Kanchipuram'
]

# Example usage based on user input
user_choice = input("Enter '1' to find allotted institute based on rank, quota, and category, \n'2' to find required rank for institute, quota, and category, or \n'3' to find all possible colleges for a given rank, quota, and category: ")

if user_choice == '1':
    rank_example = input("Enter the rank of the student: ")
    print("Select from the following quotas:")
    for i, quota in enumerate(quota_options, start=1):
        print(f"{i}. {quota}")
    quota_choice = int(input("Enter your choice (1-15): "))
    quota_example = quota_options[quota_choice - 1]
    print("Select from the following categories:")
    for i, category in enumerate(category_options, start=1):
        print(f"{i}. {category}")
    category_choice = int(input("Enter your choice (1-10): "))
    category_example = category_options[category_choice - 1]
    predicted_institute = predict_allotted_institute(rank_example, quota_example, category_example)
    if predicted_institute != "No matching institute found":
        print("Predicted Institute:", predicted_institute)
    else:
        print("No matching institute found")

elif user_choice == '2':
    print("Select from the following institutes:")
    for i, institute in enumerate(institute_options, start=1):
        print(f"{i}. {institute}")
    institute_choice = int(input("Enter your choice (1-430): "))  # Adjust the range based on the number of institutes
    institute_example = institute_options[institute_choice - 1]
    print("Select from the following quotas:")
    for i, quota in enumerate(quota_options, start=1):
        print(f"{i}. {quota}")
    quota_choice = int(input("Enter your choice (1-15): "))
    quota_example = quota_options[quota_choice - 1]
    print("Select from the following categories:")
    for i, category in enumerate(category_options, start=1):
        print(f"{i}. {category}")
    category_choice = int(input("Enter your choice (1-10): "))
    category_example = category_options[category_choice - 1]
    predicted_rank = predict_rank_for_institute(institute_example, quota_example, category_example)
    print(f"To get admitted to {institute_example} under {quota_example} quota, you need to achieve rank:", predicted_rank)

elif user_choice == '3':
    rank_example = input("Enter the rank of the student: ")
    print("Select from the following quotas:")
    for i, quota in enumerate(quota_options, start=1):
        print(f"{i}. {quota}")
    quota_choice = int(input("Enter your choice (1-15): "))
    quota_example = quota_options[quota_choice - 1]
    print("Select from the following categories:")
    for i, category in enumerate(category_options, start=1):
        print(f"{i}. {category}")
    category_choice = int(input("Enter your choice (1-10): "))
    category_example = category_options[category_choice - 1]
    possible_colleges = find_possible_colleges(rank_example, quota_example, category_example)
    if possible_colleges is not None:
        print("Possible Colleges:")
        for index, row in possible_colleges.iterrows():
            print(f"Rank: {row['Rank']}, Institute: {row['Allotted Institute']}")
    else:
        print("No matching colleges found")

else:
    print("Invalid choice. Please enter '1', '2', or '3'.")


#USING SQL SERVER

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# Function to connect to MySQL database and fetch data
def load_data_from_mysql(host, database, user, password, table_name):
    try:
        # Connect to MySQL database
        conn = mysql.connector.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )

        if conn.is_connected():
            print('Connected to MySQL database')

            # Query to fetch data from a specific table
            query = f"SELECT * FROM {table_name};"

            # Load data into Pandas DataFrame
            df = pd.read_sql(query, conn)

            return df

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")

    finally:
        # Close the database connection
        if 'conn' in locals() and conn.is_connected():
            conn.close()
            print('Connection closed.')

# Replace these with your MySQL connection details
host = 'localhost'
database = 'new_allottedlist'
user = 'root'
password = 'Sanjay@2024'
table_name = 'allotedlist'

# Load data from MySQL into a DataFrame
df = load_data_from_mysql(host, database, user, password, table_name)
if df is not None:
    print(df)
else:
    print('Failed to load data from MySQL.')

# Load model and tokenizer
model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

# Function to create context from data
def create_context(data):
    context = ""
    for index, row in data.iterrows():
        context += f"Rank: {row['Rank']}, Quota: {row['Allotted Quota']}, Institute: {row['Allotted Institute']}, Category: {row['Allotted Category']}. "
    return context

# Define the function to get the prediction
def get_prediction(question, context):
    QA_input = {
        'question': question,
        'context': context
    }
    res = nlp(QA_input)
    return res['answer']

# Function to predict the allotted institute based on rank, quota, and category
def predict_allotted_institute(rank, quota, category):
    filtered_data = data[(data['Rank'] == int(rank)) & (data['Allotted Quota'] == quota) & (data['Allotted Category'] == category)]
    if not filtered_data.empty:
        institute = filtered_data['Allotted Institute'].values[0]
        return institute
    else:
        return "No matching institute found"

# Function to predict the rank required for a specific institute, quota, and category
def predict_rank_for_institute(institute, quota, category):
    filtered_data = df1[(df['Allotted Institute'] == institute) & (df['Allotted Quota'] == quota) & (df['Allotted Category'] == category)]
    if not filtered_data.empty:
        return filtered_data['Rank'].min()  # Assuming you want the best rank for the institute
    else:
        return "No matching rank found"

# Function to find all possible colleges for a given rank, quota, and category
def find_possible_colleges(rank, quota, category):
    filtered_data = df[(df['Rank'] <= int(rank)) & (df['Allotted Quota'] == quota) & (df['Allotted Category'] == category)]
    if not filtered_data.empty:
        unique_colleges = filtered_data.groupby('Allotted Institute')['Rank'].min().reset_index()
        sorted_colleges = unique_colleges.sort_values(by='Rank')
        return sorted_colleges
    else:
        return None

# List of all quota options
quota_options = [
    'Open Seat Quota',
    'All India',
    'Delhi University Quota',
    'IP University Quota',
    'Deemed/Paid Seats Quota',
    'Internal - Puducherry UT Domicile',
    'Aligarh Muslim University (AMU) Quota',
    'Delhi NCR Children/Widows of Personnel of the Armed Forces (CW) Quota',
    'Employees State Insurance Scheme (ESI)',
    'Foreign Country Quota',
    'Non-Resident Indian',
    'Muslim Minority Quota',
    'Jain Minority Quota',
    'Non-Resident Indian(AMU)Quota',
    'Employees State Insurance Scheme(ESI)'
]

# List of all category options
category_options = [
    'Open',
'OBC',
'SC',
'EWS',
'ST',
'Open PwD',
'OBC PWD',
'EWS PWD',
'SC PwD',
'ST PwD'
]

# List of all institute options
institute_options = [
    'JIPMER Puducherry',
'AIIMS, New Delhi',
'Institute of PG Medical Educational and Research, Kolkata',
'Madras Medical College, Chennai',
'AIIMS, Bhubaneswar',
'Atal Bihari Vajpayee Institute of Medical Sciences and Dr RML Hospital',
'Seth G.S. Medical College, Mumbai',
'Maulana Azad Medical College',
'Vardhman Mahavir Medical College and Safdarjung Hospital',
'AIIMS, Jodhpur',
'M.P. Shah Medical College, Jamnagar',
'University College of Medical Sciences',
'S.M.S. Medical College, Jaipur',
'Government Medical College And Hospital, Chandigarh',
'AIIMS, Bhopal',
'AIIMS, Rishikesh',
'Jawaharlal Nehru Medical College, AMU, Aligarh',
'Government Medical College Baroda, Anandpura',
'Institute of Medical Sciences, Varanasi',
'Lady Hardinge Medical College (Female Seat only)',
'AIIMS, Nagpur',
'Government Medical College, Thiruvananthapuram',
'Dr. B.S.A. Medical College',
'Government Medical College, Surat',
'B.J. Medical College, Ahmedabad',
'AIIMS Mangalagiri',
'Kasturba Medical College, Manipal',
'Government Medical College, Kottayam',
'Bangalore Medical College and Research Institute',
'Gandhi Medical College, Secunderabad',
'Karnatak Institute Of Medical Science, Hubli',
'B.J. Government Medical College, Pune',
'AIIMS, Patna',
'KGMC, Lucknow',
'AIIMS, Kalyani',
'Medical College, Kolkata',
'Indira Gandhi Medical College',
'AIIMS, Raipur',
'Government Medical College, Srinagar',
'AIIMS, Bibi Nagar',
'Government Medical College, Kozhikode',
'Coimbatore Medical College, Peelamedu',
'AIIMS Rajkot',
'Lokmanya Tilak Municipal Medical College, Mumbai',
'Madurai Medical College, Madurai',
'NDMC Medical College',
'AIIMS Bathinda',
'Stanley Medical College, Chennai',
'Grant Medical College and Sir J.J.Hospital, Mumbai',
'AIIMS Guwahati',
'JIPMER Karaikal',
'Belgaum Institute Of Medical Science, Belagavi',
'AIIMS Bilaspur Changar Palasiyan',
'Sardar Patel Medical College, Bikaner',
'S.C.B. Medical College, Cuttack',
'Indira Gandhi Institute of Medical Sciences, Patna',
'AIIMS, Gorakhpur',
'R.N.T. Medical College, Udaipur',
'Dr. Ram Manohar Lohia Institute of Medical Science, Lucknow',
'L.L.R.M. Medical College, Meerut',
'M.G.M. Medical College, Indore',
'AIIMS, Rai Bareli',
'Government Medical College, Thrissur',
'DR.S.N. Medical College, Jodhpur',
'Pt. D.D.U Medical College, Rajkot',
'T.D. Medical College, Allappuzha',
'Mysore Medical and Research Institute, Mysore',
'Pt. B.D. Sharma PGIMS',
'Government Medical College, Patiala',
'Dr. B.R. Ambedkar State Institute of Medical Sciences',
'AIIMS Jammu',
'Government Medical College, Manjeri',
'Netaji Subhash Chandra Bose Mc, Jabalpur',
'Guntur Medical College, Guntur',
'Topiwala National Medical College, Mumbai',
'Patna Medical College',
'Kurnool Medical College, Kurnool',
'Osmania Medical College, Koti, Hyderabad',
'Jawaharlal Nehru Medical, Ajmer',
'Government Medical College, Kota',
'Atal Bihari Vajpayee Government Medical College, Vidisha',
'Shri Atal Bihari Vajpayee Medical College and Research Institute, Bengaluru',
'Nilratan Sirkar Medical College, Kolkata',
'Government Kilpauk Medical College, Chennai',
'RUHS College of Medical Sciences, Jaipur',
'Government Medical College, Ernakulam',
'AIIMS, Deoghar',
'Hinduhridayasamrat Balasaheb Thackeray Medical College and Dr. R. N. Cooper Municipal General Hospital, Juhu',
'Gajra Raja Medical College, Gwalior',
'Siddartha Medical College, Vijayawada',
'Calcutta National Medical College, Kolkata',
'ESIC Medical College, Faridabad',
'Goa Medical College, Panaji',
'ESI-MC and PGIMS and R, Bangalore',
'Government Medical College, Aurangabad',
'G.S.V.M. Medical College, Kanpur',
'Maharaja K.C. Gajapati Medical College, Brahmapur',
'Indira Gandhi Medical College and RI, Puducherry',
'ESIC Medical College, Hyderabad',
'Government Vellore Medical College, Vellore',
'Government Medical College, Nagpur',
'Rajendra Institute of Medical Sciences, Ranchi',
'Pt. J N M Medical College, Raipur',
'AIIMS, Madurai',
'Andhra Medical College, Visakhapatnam',
'Indira Gandhi Government Medical College, Nagpur',
'S.N. Medical College, Agra',
'Maharani Laxmi Bai Medical College, Jhansi',
'Government Medical College, Omandurar',
'M.G.M. Medical College, Jamshedpur',
'DR.Rajendra Prasad Medical College, Tanda',
'R G Kar Medical College',
'Dr.S.C. Government Medical College, Nanded',
'Government Institute of Medical Sciences, Greater Noida',
'Government Medical College, ESIC, Kollam',
'Government Medical College and ESIC Hospital, Coimbatore',
'Moti Lal Nehru Medical College, Allahabad',
'Government medical college, Amritsar',
'Gandhi Medical College, Bhopal',
'MG Institute of Medical Sciences, Sevagram Wardha',
'Government Medical College, Jammu',
'Rangaraya Medical College, Kakinada',
'Guwahati Medical College, Guwahati',
'K.A.P. Viswanatham Government Medical College, Tiruchirapalli',
'Rajiv Gandhi Medical College, Thane',
'Gulbarga Institute of Medical Sciences',
'Government Doon Medical College, Dehradun',
'Government Medical College, Tirunelveli',
'V.S.S. Medical College, Burla',
'Government Medical College, Kanker',
'Guru Govind Singh Medical College',
'B.R.D. Medical College, Gorakhpur',
'Government Medical College, Palakkad',
'Uttar Pradesh University of Medical Sciences, Saifai',
'Kalpana Chawla Government Medical College',
'Government Medical College, Miraj',
'Nalanda Medical College, Patna',
'Rajarshee Chhatrapati Shahu Maharaj Government Medical College Kolhapur',
'Vijayanagar Institute of Medical Science, Bellary',
'SLBS Government Medical College, Mandi',
'Uttaranchal F Hospital Trust Medical College, Haldwani',
'Andaman and Nicobar Islands Institute of Medical Science',
'NEIGRIHMS, Shillong',
'Mandya Institute Of Medical Science, Mandya',
'Medical College, Bhavnagar',
'Sagar Dutta Medical College and Hospital',
'Regional Inst Of Medical Sci, Imphal',
'Government Medical College, Kannur',
'Dr. YS Parmar Government Medical College, Sirmaur',
'Jhalawar Medical College, Jhalawar',
'Barasat Government Medical College and Hospital',
'Government Medical College, Idukki',
'Jawaharlal Nehru Medical College, Bhagalpur',
'Vardhman Institute Of Medical Sciences, Nalanda',
'ACSR Government Medical College, Nellore',
'North Bengal Medical College, Darjeeling',
'Employees State Insurance Corporation Medical College, Alwar',
'Karwar Institute of Medical Sciences, Karwar',
'Darbhanga Medical College, Laheriasarai',
'ESIC Medical College and PGIMSR, Chennai',
'Government Medical College, Pali',
'Government Medical College Baramati',
'Shaheed Nirmal Mahto Medical College and Hospital, Dhanbad',
'Shri Kalyan Government Medical College, Bajaj Gram',
'Sri Venkateswara Medical College, Tirupati',
'Government Medical College, Bhilwara',
'Burdwan Medical College, Burdwan',
'Government Medical College, Konni',
'ESIC PGIMSR, Joka',
'SJP Medical College, Bharatpur',
'Government Medical College and Super Facility Hospital, Azamgarh',
'Government Medical College, Churachandpur',
'Chengalpattu Medical College, Kancheepuram',
'Bankura Sammilani Medical College, Lokepur',
'College of Medicine and JNM Hospital, Kalyani',
'Dr. Radhakrishnan Government Medical College, Hamirpur',
'Government medical college, Dausa Rajasthan',
'BPS Government Medical College, Sonepat (Female Seat only)',
'Government Medical College, Dungarpur',
'Government Medical college, Sriganganagar',
'Hassan Institute Medical Sciences, Hassan',
'Kakatiya Medical College, Warangal',
'Silchar Medical College, Silcher',
'Shri Atal Bihari Vajpayee Government Medical College',
'Chhattisgarh Institute of Medical Sciences, Bilasp',
'Bundelkhand Medical College, Sagar',
'Mursidabad Medical College and Hospital, Mursidabad',
'Rajkiya Medical College, Jalaun',
'Autonomous State Medical College, Siddharthnagar',
'Thanjavur Medical College,Thanjavur',
'Dr. Vaishampayam Memorial Medical College, Sholapur',
'SUH Maulana Mahmood Hasan Medical College, Saharanpur',
'Pt. Jawahar Lal Nehru Government Medical College, Chamba',
'Assam Medical College, Dibrugarh',
'Government Medical College Chittorgarh',
'Sri Krishna Medical College, Muzaffarpur',
'S.S. Medical College, Rewa',
'Government Medical College Firozabad,Daulattapur',
'Veer Chandra Singh Garhwali Government Institute of Medical Science and Research,Badrinath Marg',
'SKIMS Medical College, Bemina',
'Kodagu Institute of Medical Sciences, Kodagu',
'Government Medical College Faizabad',
'Shimoga Institute Of Medical Science, Shimoga',
'Jorhat Medical College and Hospital',
'College Government Medical College Alwar',
'Midnapore Medical College, Midnapur',
'Government Medical College And General Hospital, Satara',
'Rajiv Gandhi Institute Of Medical Sciences, Kadapa',
'Autonomous State Medical College Society , Mirzapur',
'Government Medical College Mahasamund',
'Anugrah Narayan Magadh Medical College, Gaya',
'ESIC Medical College, Gulbarga',
'ESIC Medical College and Hospital, Bihta',
'MRA Medical College , Ambedkar nagar',
'Soban Singh Jeena Government Institute of Medical Science and Research, Almora',
'Kanyakumari Government Medical College,Asaripallam',
'Government Medical College, Akola',
'Government Medical College Barmer Rajasthan',
'Government Medical College, Churu',
'Government Mohan Kumaramangalam Medical College, Salem',
'Government Medical College, Kannauj',
'Goverment Medical College, Datia',
'Government Medical College Basti,Rampur',
'Theni Government Medical College, Theni',
'Government Medical College Ratlam,Gram Banjali',
'Raichur Institute Of Medical Science, Raichur',
'Government Medical College Karimnagar',
'Shaheed Hasan Khan, Mewati Government Medical College',
'Government Erode Medical College Perundurai',
'Government Medical College Badaun',
'Autonomous State Medical College, Pratapgarh',
'UNS Autonomous State Medical Colleges, Jaunpur',
'Government Medical College Nalgonda',
'Government Medical College Satna',
'Government Medical College, Thiruvallur',
'Sh Vasant Rao Naik Government Medical College ,Yavatmal',
'Government medical college Karauli',
'Sri Jagannath Medical College and Hospital, Puri',
'Gvmc, Villupuram',
'Government Medical College, Latur',
'Lt. B R K Government Medical College, Jagdalpur',
'Diamond Harbour Government Medical College,New Town',
'Government Medical College, Nilgiris',
'Saheed Laxman Nayak Medical College and Hospital, Koraput',
'Bidar Institute of Medical Science',
'Government Medical College and Hospital, Chandrapur',
'Government Medical College, Dholpur',
'Government Medical college Shivpuri',
'Chhindwara Institute of Medical Sciences, Chhindwara',
'Government Pudukkottai Medical College Hospital, Pudukkottai',
'Rajah Muthiah Medical College, Annamalai',
'Government Medical College, Dindigul',
'Government Medical College, Sirohi',
'Thoothukudi Medical College,Thoothukudi',
'Autonomous State Medical College ,Etah',
'Government medical college, Shahjhanpur',
'Fakhruddin Ali Ahmed Medical College, Barpeta',
'Government Medical College, Khandwa',
'Rani Durgavati Medical College, Banda',
'Goverment Medical College, Bettiah',
'Government Dharamapuri Med Coll,Dharmapuri',
'Shri Bhausaheb Hire Government Medical College, Dhule',
'Hamdard Inst of Med Sciences and Research',
'Goverment Medical College And Hospital Jalgaon',
'Maharshi Devraha Baba Autonomous State. Medical College, Deoria',
'Goverment Medical College Nandurbar',
'Sheikh Bhikhari Medical College and Hospital, Hazaribag ( Formerly Called As- Hazaribagh Medical College, Hazaribag)',
'Government Medical College, Ananthapuram',
'Government Medical College, Tiruppur',
'Government Medical College, Krishnagiri',
'Phulo Jhano Medical College , Dumka',
'Malda Medical College',
'Lt. L A M Government Medical College, Raigarh',
'Government Medical College, Gondia',
'SVIMS - Sri Padmavathi Medical College for Women, Tirupati (Female Seat only)',
'Bhima Bhoi Medical College and Hospital , Balangir',
'Chandulal Chandrakar Memorial Government Medical College, Durg',
'Government Medical College and District General Hospital,Ratnagiri',
'Government medical College, Namakkal',
'Autonomous State Medical college Society Hardoi,Gaura Danda',
'Agartala Government Medical College,Agartala',
'Government Medical College and Hospital, Alibag-Raigad',
'Medinirai Medical College (previously Known as Palamu Medical College)',
'Deben Mahata Government Medical College and Hospital,Vill Hatuara',
'Autonomous State Medical College , Fatehpur',
'Government Medical College Karur',
'Rajiv Gandhi Institute Medical Science of Adilabad',
'Swami Ramanand Tirth Rural Medical College,Ambajogai',
'Government Medical College and Hospital, Jalpaiguri',
'Rampurhat Government Medical College Rampurhat',
'GMERS Medical College, Porbandar',
'GMERS Medical College, Navsari',
'Government Thiruvarur medical college, Thiruvarur',
'Jannayak Karpoori Thakur Medical college and Hospital, Madhepura',
'Gadag Institute of Medical Sciences',
'Maharaja Jitendra Narayan Medical College and Hospital Coochbehar',
'Government of Medical College and Hospital, Balasore',
'Government Medical College, Kathua',
'Chikkaballapura Institute of Medical Sciences',
'Raiganj Government Medical College',
'Lakhimpur Medical College',
'Government Medical College, Korba',
'Sarat Chandra Chattopadhyay Government Medical College and Hospital, Uluberia',
'JLN IMS, IMPHAL',
'Government Medical College Shahdol',
'Autonomous State Medical College, Ghazipur',
'Government Medical College, Rajnandgaon',
'Saheed Rendo Majhi Medical College and Hospital, Bhawanipatna',
'Tezpur Medical College, Tezpur',
'Government Sivgangai Medical College Sivagangai',
'Government Medical College Purnea',
'Pt. Raghunath Murmu Medical College, Baripada',
'GMERS Medical College, Morbi',
'Government Medical College Jayashankar Bhupalpally',
'GMERS Medical College, Panchmahal Godhra',
'Thiruvannamalai Medical College, Thiruvannamalai',
'Government Medical College, Osmanabad',
'Dhubri Medical College',
'Government Medical College, Mahabubangar',
'Government Medical College, Virudhunagar',
'Government Medical College, Doda',
'Government medical college Jangaon',
'Government Medical College Rajanna Sircilla',
'Rajmata shrimati devendra kumari singhdeo government medical college, ambikapur',
'Government Medical College Siddipet',
'Government Medical College, Ramanathapuram',
'Haveri Institute of Medical Sciences, Karnataka',
'Prafulla Chandra Sen Government Medical College and Hospital, Arambagh',
'RIMS Srikakulam',
'Government Medical College, Parbhani',
'Government Medical College, Wanaparthy',
'Government Medical College, Baramulla',
'Government Medical College Khaleelwadi',
'Government Medical College, Ariyalur',
'Government Medical College and Hospital, Keonjhar',
'Government Medical College, Kheora',
'Government Medical College, Nagarkurnool',
'Government Medical College, Kallakurichi',
'C. Institute of Medical Sciences, Chamarajanagar',
'Nalbari Medical College and Hospital Dakhingaon',
'Government Medical College, Handwara',
'RIMS, Ongole',
'Koppal Institute of Medical Sciences',
'GMERS Medical College, Rajpipla',
'Government Medical College, Sindhudurg',
'Government Medical College, Mancherial',
'Tamralipto Government Medical College and Hospital',
'Jhargram Government Medical College and Hospital',
'Government Medical College Machilipatnam',
'Nagaland Institute Of Medical Science And Research Phirebagie',
'Government Medical College, Khammam',
'Chikkamagaluru Institute of Medical Sciences',
'Government Medical College, Eluru',
'Zoram Medical College',
'Government Medical College, Mahabubabad',
'Government Medical College Suryapet',
'Yadgiri Institute Of Medical Sciences, Yadgiri',
'Tomo Riba Institute Health and Medical Sciences, Naharlagun',
'Diphu Medical College and Hospital',
'Government Medical College, Sundargarh',
'Government Medical College, Sangareddy',
'Government Medical College, Nagapattinam',
'Government medical college Kamareddy',
'Nagaon Medical college,Dipholu',
'Government Medical College Nandyal',
'Government Medical College Vikarabad',
'Kokrajhar Medical College and Hospital Rangalikhata',
'Government medical college, Rajamahendravaram',
'Government Medical College, Jagtial',
'Government Medical College, Ramagundam',
'Government Medical College, Bhadradri',
'Government Medical College Vizianagaram',
'Kasturba Medical College, Mangalore',
'Government Medical College Nirmal',
'Sri Ramachandra Medical College and Research Institute, Chennai',
'Government Medical College, Kumuram Bheem Asifabad',
'Amrita School of Medicine Faridabad',
'Symbiosis Medical College for Women, Pune (Female Seat only)',
'JLN Medical College, Datta Meghe',
'MGM Medical College, Navi Mumbai',
'Yenepoya Medical College, Mangalore',
'Manipal Tata Medical College, Jamshedpur',
'Amrita Institute of Medical Science, Kochi',
'JSS Medical College, Mysuru',
'SBKS Medical Institute and Research Centre, Sumandeep Vidyapeeth',
'Jawahar Lal Nehru Medical College, Belagavi',
'Kalinga Institute of Medical Sciences, Bhubaneswar',
'M.M Institute of medical and Research',
'K.S Hegde Medical Academy, Mangaluru',
'MGM Medical College, Aurangabad',
'Santosh Medical College and Hospital',
'Bharati Vidyapeeth DU Medical College,Dhankawadi',
'Rural Medical College and PIMS, Loni',
'Dr. DY Patil Medical College and Hospital, Pune',
'Raja Rajeswari Medical College Bengaluru',
'Dr. DY Patil Medical College, Navi mumbai',
'Institute of Medical Sciences and SUM Hospital, Bhubaneswar',
'Aarupadai Veedu Medical College and Hospital',
'Sree Balaji Medical College and Hospital, Chennai',
'Jagadguru Gangadhar Mahaswamigalu Moorusavirmath Medical College, Hubballi',
'Datta Meghe Medical College, Wanadongri',
'Krishna Institute of Medical Science, Karad',
'Mahatma Gandi Mission Medical College, vashi',
'Dr. DYP Education Society Deemed University, Kolhapur',
'SDU Medical College, Kolar',
'B.L.D.E University, Bijapur',
'Bhaarath Medical College And Hospital, Chennai',
'Saveetha Medical College, Chennai',
'Sri Lakshmi Narayana Institute of Medical Science',
'Sri Siddhartha Medical College DU, Tumkur',
'SRM Medical College and Hospital, Chennai',
'VMKV Medical College and Hospital, Salem',
'Sri Lalithambigai Medical College and Hospital',
'Chettinad Hospital and Research Institute, Kancheepuram',
'BV Deemed University Medical College and Hospital, Sangli',
'GITAM Institue of Medical Science and Research, Visakhapatnam',
'Mahatma Gandhi Medical College, Pondicherry',
'Vinayaka Missions Medical College and Hospital, Karaikal',
'VELS Medical College and Hospital, Tiruvallur',
'ACS Medical College and Hospital, Chennai',
'Sri Siddhartha institute of Medical Science and Research Centre, T Begur',
'Shri Sathya Sai Medical College and RI, Chennai',
'NAMO Medical Education and Research Institute, Govt Medical College Silvassa',
'Government Medical College, Hanumangarh',
'Government medical college, Bahraich',
'Government Medical College, Bundi',
'Government Medical College, Udhampur',
'Government Medical College, Anantnag',
'Meenakshi Medical College Hospital and Research Institute, Kanchipuram'
]

# Example usage based on user input
user_choice = input("Enter '1' to find allotted institute based on rank, quota, and category, \n'2' to find required rank for institute, quota, and category, or \n'3' to find all possible colleges for a given rank, quota, and category: ")

if user_choice == '1':
    rank_example = input("Enter the rank of the student: ")
    print("Select from the following quotas:")
    for i, quota in enumerate(quota_options, start=1):
        print(f"{i}. {quota}")
    quota_choice = int(input("Enter your choice (1-15): "))
    quota_example = quota_options[quota_choice - 1]
    print("Select from the following categories:")
    for i, category in enumerate(category_options, start=1):
        print(f"{i}. {category}")
    category_choice = int(input("Enter your choice (1-10): "))
    category_example = category_options[category_choice - 1]
    predicted_institute = predict_allotted_institute(rank_example, quota_example, category_example)
    if predicted_institute != "No matching institute found":
        print("Predicted Institute:", predicted_institute)
    else:
        print("No matching institute found")

elif user_choice == '2':
    print("Select from the following institutes:")
    for i, institute in enumerate(institute_options, start=1):
        print(f"{i}. {institute}")
    institute_choice = int(input("Enter your choice (1-430): "))  # Adjust the range based on the number of institutes
    institute_example = institute_options[institute_choice - 1]
    print("Select from the following quotas:")
    for i, quota in enumerate(quota_options, start=1):
        print(f"{i}. {quota}")
    quota_choice = int(input("Enter your choice (1-15): "))
    quota_example = quota_options[quota_choice - 1]
    print("Select from the following categories:")
    for i, category in enumerate(category_options, start=1):
        print(f"{i}. {category}")
    category_choice = int(input("Enter your choice (1-10): "))
    category_example = category_options[category_choice - 1]
    predicted_rank = predict_rank_for_institute(institute_example, quota_example, category_example)
    print(f"To get admitted to {institute_example} under {quota_example} quota, you need to achieve rank:", predicted_rank)

elif user_choice == '3':
    rank_example = input("Enter the rank of the student: ")
    print("Select from the following quotas:")
    for i, quota in enumerate(quota_options, start=1):
        print(f"{i}. {quota}")
    quota_choice = int(input("Enter your choice (1-15): "))
    quota_example = quota_options[quota_choice - 1]
    print("Select from the following categories:")
    for i, category in enumerate(category_options, start=1):
        print(f"{i}. {category}")
    category_choice = int(input("Enter your choice (1-10): "))
    category_example = category_options[category_choice - 1]
    possible_colleges = find_possible_colleges(rank_example, quota_example, category_example)
    if possible_colleges is not None:
        print("Possible Colleges:")
        for index, row in possible_colleges.iterrows():
            print(f"Rank: {row['Rank']}, Institute: {row['Allotted Institute']}")
    else:
        print("No matching colleges found")

else:
    print("Invalid choice. Please enter '1', '2', or '3'.")


In [ ]:
pip install sqlalchemy mysql-connector-python pandas

In [ ]:
from transformers import pipeline